# [MIPSGAL](https://irsa.ipac.caltech.edu/data/SPITZER/MIPSGAL/) のデータを一括ダウンロード
- 引数1: int 型の 0-359 数字 (間に観測対象外のものも含まれるため注意)
- 引数2: str 型の 'p' か 'n'
- 引数3: str 型で保存場所の Path

In [ ]:
import sys
import pathlib
import numpy
import astropy.io.fits
import urllib.request

sys.path.append('..')

### 指定したファイルをダウンロード可能か試験

In [ ]:
save_path = pathlib.Path('~/data/spitzer/mipsgal/').expanduser()
url = 'https://irsa.ipac.caltech.edu/data/SPITZER/MIPSGAL/images/mosaics24/'
file_name = 'MG0000n005_024.fits'
with urllib.request.urlopen(url+file_name) as u:
    with open(save_path/file_name, 'bw') as o:
        o.write(u.read())

### 以下、作成した関数および使用例

In [ ]:
def download_mipsgal_data(l, b, save_path):
    '''
    l must be int from 0 to 359.
    b must be str, 'p' or 'n'.
    '''
    save_path = pathlib.Path(save_path).expanduser()
    url = 'https://irsa.ipac.caltech.edu/data/SPITZER/MIPSGAL/images/mosaics24/'
    l = str(l)+'0'
    while len(l)!=4:
        l = '0'+l
        
    file_name = 'MG'+l+b+'005_024.fits'
    with urllib.request.urlopen(url+file_name) as u:
        with open(save_path/file_name, 'bw') as o:
            o.write(u.read())
            
    return

# 2020/03/29 追記
# mipsgal には 293 のデータがないので、全て 0 の fits を作り出す関数を作った。
# download_mipsgal_data を叩いた後に使用しましょう。
def make_mipsgal_293data(b, save_path):
    '''
    b must be str, 'p' or 'n'.
    '''
    path = pathlib.Path(save_path)    
    file_dict = {
        0: (path/'MG2930{}005_024.fits'.format(b)).expanduser(),
        1: (path/'MG2940{}005_024.fits'.format(b)).expanduser(),
        2: (path/'MG2950{}005_024.fits'.format(b)).expanduser(),
    }
    files = file_dict[1], file_dict[2]
    hdus = astropy.io.fits.open(files[0])[0], astropy.io.fits.open(files[1])[0]
    
    header, data = hdus[0].header, hdus[0].data
    d_l = hdus[1].header['CRVAL1'] - hdus[0].header['CRVAL1']
    
    header['CRVAL1'] = header['CRVAL1'] - d_l
    header['FILENAME'] = file_dict[0].name
    data = numpy.nan_to_num(data)
    data = numpy.where(data>0.0, 0.0, 0.0)
    data = data.astype(numpy.float32)
    new_hdu = astropy.io.fits.PrimaryHDU(data, header)
    new_hdul = astropy.io.fits.HDUList([new_hdu])
    new_hdul.writeto(file_dict[0], overwrite=True)
    
    return

In [ ]:
save_path = '~/data/spitzer/mipsgal/mips24'

l = [i for i in range(0, 66)]
for i in l:
    download_mipsgal_data(i, 'p', save_path)
    download_mipsgal_data(i, 'n', save_path)
    
l = [i for i in range(294, 359)]
for i in l:
    download_mipsgal_data(i, 'p', save_path)
    download_mipsgal_data(i, 'n', save_path)

# 2020/03/29 追記
make_mipsgal_293data('p', save_path)
make_mipsgal_293data('n', save_path)

### /src/utils/data_utils.py に関数を追加した。

In [ ]:
from src.utils import data_utils
save_path = '~/data/spitzer/mipsgal/mips24'

data_utils.download_mipsgal_data(295, 'p', save_path)
data_utils.download_mipsgal_data(294, 'p', save_path)
data_utils.download_mipsgal_data(295, 'n', save_path)
data_utils.download_mipsgal_data(294, 'n', save_path)

data_utils.make_mipsgal_293data('p', save_path)
data_utils.make_mipsgal_293data('n', save_path)